In [283]:
import pandas as pd

In [295]:
# from Baseball Savant, get weighted launch angle using TY and LY data
# min pa = 50
def read_launch_angle(year):
    df = pd.read_csv(f'Launch Angle {year - 1}-{year}.csv', index_col=False).drop('Unnamed: 6', axis=1)
    
    # for LY data, give it half-weight
    df.loc[df['year'] < year, 'b_total_pa'] /= 2
    
    # calculate the weighted launch angle
    df['weight'] = df['b_total_pa'] * df['launch_angle_avg']
    df_grouped = df.groupby(['player_id']).sum()
    df_grouped['launch_angle_avg'] = df_grouped['weight'] / df_grouped['b_total_pa']
    df_grouped = df_grouped[df_grouped['year'] >= year]
    
    # join with player ids
    player_map = pd.read_csv('../PLAYERIDMAP 2022.csv')[['MLBID', 'PLAYERNAME']].set_index('MLBID')
    merge = df_grouped[['launch_angle_avg']].merge(player_map, how='left', left_index=True, right_index=True)
    
    return merge[['PLAYERNAME', 'launch_angle_avg']]

la_data = read_launch_angle(2022)
la_data.head()

,PLAYERNAME,launch_angle_avg
405395,Albert Pujols,15.094990
408234,Miguel Cabrera,6.889368
425877,Yadier Molina,16.152912
429664,Robinson Cano,5.800000
435559,Kurt Suzuki,19.946903


In [285]:
# TRAJECTORY CONVERSIONS:
# 1 if LA <= 10
# 2 if 10 < LA <= 17
# 3 if 17 < LA <= 21.5
# 4 if 21.5 < LA
trajectory_angle_conversion = {
    2: 5,
    3: 15,
    4: 20
}
print(len(la_data[la_data['launch_angle_avg'] <= 5]))
print(len(la_data[(la_data['launch_angle_avg'] > 5) & (la_data['launch_angle_avg'] <= 15)]))
print(len(la_data[(la_data['launch_angle_avg'] > 15) & (la_data['launch_angle_avg'] <= 20)]))
print(len(la_data[la_data['launch_angle_avg'] > 20]))

39
329
146
34


In [286]:
def get_traj(la):
    current_traj = 1
    while current_traj < 4:
        floor = trajectory_angle_conversion[current_traj + 1]
        if la > floor:
            current_traj += 1
        else:
            break
    return current_traj

In [287]:
def assign_traj(year):
    la_data = read_launch_angle(2022)
    la_data['Trajectory'] = la_data.apply(lambda x: get_traj(x['launch_angle_avg']), axis=1)
    return la_data[['PLAYERNAME', 'Trajectory']].dropna()

traj_df = assign_traj(2022)
traj_df.head()

,PLAYERNAME,Trajectory
405395,Albert Pujols,3
408234,Miguel Cabrera,2
425877,Yadier Molina,3
429664,Robinson Cano,2
435559,Kurt Suzuki,3


In [293]:
traj_df.to_csv('2022_Trajectory.csv')